<a href="https://colab.research.google.com/github/anupamatagde-debug/j/blob/main/quickstarts/Grounding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2026 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Gemini API: Getting started with information grounding for Gemini models

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Grounding.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In this notebook you will learn how to use information grounding with [Gemini models](https://ai.google.dev/gemini-api/docs/models/).

Information grounding is the process of connecting these models to specific, verifiable information sources to enhance the accuracy, relevance, and factual correctness of their responses. While LLMs are trained on vast amounts of data, this knowledge can be general, outdated, or lack specific context for particular tasks or domains. Grounding helps to bridge this gap by providing the LLM with access to curated, up-to-date information.

Here you will experiment with:
- Grounding information using <a href="#search_grounding">Google Search grounding</a>
- Grounding real-world information using <a href="#maps_grounding">Google Maps grounding</a>
- Adding <a href="#yt_links">YouTube links</a> to gather context information to your prompt
- Using <a href="#url_context">URL context</a> to include website, pdf or image URLs as context to your prompt

## Set up the SDK and the client

### Install SDK

This guide uses the [`google-genai`](https://pypi.org/project/google-genai) Python SDK to connect to the Gemini models.

In [3]:
# Grounding with Google Maps was introduced in 1.43
%pip install -q -U "google-genai>=1.43.0"

### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](../quickstarts/Authentication.ipynb) quickstart for an example.

In [4]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

### Select model and initialize SDK client

Select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [5]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "gemini-3-flash-preview" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-2.5-flash-preview", "gemini-3-pro-preview"] {"allow-input":true, isTemplate: true}

<a name="search_grounding"></a>

## Use Google Search grounding

Google Search grounding is particularly useful for queries that require current information or external knowledge. Using Google Search, Gemini can access nearly real-time information and better responses.

To enable Google Search, simply add the `google_search` tool in the `generate_content`'s `config` that way:
```
    config={
      "tools": [
        {
          "google_search": {}
        }
      ]
    },
```

In [ ]:
from IPython.display import HTML, Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What was the latest Indian Premier League match and who won?",
    config={"tools": [{"google_search": {}}]},
)

# print the response
display(Markdown(f"**Response**:\n {response.text}"))
# print the search details
print(f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}")
# urls used for grounding
print(f"Search Pages: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

**Response**:
 The latest Indian Premier League (IPL) match was the final of the IPL 2025 season, which took place on June 3, 2025. In this match, Royal Challengers Bengaluru defeated Punjab Kings by 6 runs to win their maiden title.

Search Query: ['latest Indian Premier League match and winner', 'when did IPL 2025 finish', 'IPL 2024 final match and winner']
Search Pages: olympics.com, wikipedia.org, thehindu.com, olympics.com, skysports.com, wikipedia.org, thehindu.com


You can see that running the same prompt without search grounding gives you outdated information:

In [ ]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What was the latest Indian Premier League match and who won?",
)

# print the response
display(Markdown(response.text))

The latest Indian Premier League (IPL) match was the **Final of the IPL 2024 season**.

*   **Match:** Kolkata Knight Riders (KKR) vs. Sunrisers Hyderabad (SRH)
*   **Date:** May 26, 2024
*   **Winner:** **Kolkata Knight Riders (KKR)** won by 8 wickets.

For more examples, please refer to the [dedicated notebook ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](./Search_Grounding.ipynb).

<a name="maps_grounding"></a>

## Use Google Maps grounding

Google Maps grounding allows you to easily incorporate location-aware functionality into your applications. When a prompt has context related to Maps data, the Gemini model uses Google Maps to provide factually accurate and fresh answers that are relevant to the specified location or general area.

To enable grounding with Google Maps, add the `google_maps` tool in the  `config` argument of `generate_content`, and optionally provide a structured location in the `tool_config`.

```python
client.models.generate_content(
    ...,
    config=types.GenerateContentConfig(
      # Enable the tool.
      tools=[types.Tool(google_maps=types.GoogleMaps())],
      # Provide structured location.
      tool_config=types.ToolConfig(retrieval_config=types.RetrievalConfig(
            lat_lng=types.LatLng(
                latitude=34.050481, longitude=-118.248526))),
    )
)
```

In [ ]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Do any cafes around here do a good flat white? I will walk up to 20 minutes away",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_maps=types.GoogleMaps())],
        tool_config=types.ToolConfig(
            retrieval_config=types.RetrievalConfig(
                lat_lng=types.LatLng(latitude=40.7680797, longitude=-73.9818957)
            )
        ),
    ),
)

Markdown(f"### Response\n {response.text}")

### Response
 Yes, there are several cafes around that do a good flat white within a 20-minute walk.

*   **Tiny Dancer Coffee** specifically mentions serving flat whites, along with espressos and other latte options. It's a cozy subway cafe with a 4.8-star rating and is about a 6.8-minute walk away.
*   **Solid State Coffee** is an easygoing roastery offering thoughtfully sourced brews and has a 4.7-star rating. It's approximately a 4.7-minute walk.
*   **Sote Coffee Roasters** is a warm, laid-back coffee shop serving freshly roasted brews with a 4.9-star rating, about a 5.9-minute walk.
*   **White Noise Coffee - Coffee Shop & Roastery** is an intimate cafe with globally sourced, in-house roasted beans, rated 4.7 stars, and is about a 5.0-minute walk away.
*   **Rex** offers pour-over coffee and espresso drinks and has a 4.6-star rating, located about a 4.8-minute walk from you.
*   **Thē Soirēe** is a cozy cafe featuring espresso drinks, teas, and pastries. It has a 4.7-star rating and is about a 4.4-minute walk away.
*   **Bibble & Sip** is a bakery and coffeehouse serving upscale coffees, rated 4.5 stars, and is about a 4.1-minute walk.

Search Query: ['cafes with flat white near me']


All grounded outputs require sources to be displayed after the response text. This code snippet will display the sources.

In [ ]:
def generate_sources(response: types.GenerateContentResponse):
  grounding = response.candidates[0].grounding_metadata
  # You only need to display sources that were part of the grounded response.
  supported_chunk_indices = {i for support in grounding.grounding_supports for i in support.grounding_chunk_indices}

  sources = []
  if supported_chunk_indices:
    sources.append("### Sources from Google Maps")
  for i in supported_chunk_indices:
    ref = grounding.grounding_chunks[i].maps
    sources.append(f"- [{ref.title}]({ref.uri})")

  return "\n".join(sources)


Markdown(generate_sources(response))

### Sources from Google Maps
- [Sote Coffee Roasters](https://maps.google.com/?cid=13421224117575076881)
- [Heaven on 7th Marketplace](https://maps.google.com/?cid=13100894621228039586)
- [White Noise Coffee - Coffee Shop & Roastery](https://maps.google.com/?cid=9563404650783060353)
- [Sip + Co.](https://maps.google.com/?cid=4785431035926753688)
- [Weill Café](https://maps.google.com/?cid=16521712104323291061)
- [Down Under Coffee](https://maps.google.com/?cid=3179851379461939943)

The response also includes data you can use to assemble in-line links. See the [Grounding with Google Search docs](https://ai.google.dev/gemini-api/docs/google-search#attributing_sources_with_inline_citations) for an example of this.

### Render the contextual Google Maps widget

If you are building a web-based application, you can add an interactive widget that includes a map view, the contextual location, the places Gemini considered in the query, and review snippets.

To load the widget, perform all of the following steps.
1. [Acquire a Google Maps API key](https://developers.google.com/maps/documentation/javascript/get-api-key), enabled for the Places API and the Maps JavaScript API,
1. Request the widget token in your request (with `GoogleMaps(enable_widget=True)`),
1. [Load the Maps JavaScript API](https://developers.google.com/maps/documentation/javascript/load-maps-js-api) and enable the Places library,
1. Render the [`<gmp-place-contextual/>`](https://developers.google.com/maps/documentation/javascript/reference/places-widget#PlaceContextualElement) element, setting `context-token` to the value of the `google_maps_widget_context_token` returned in the Gemini API response.

Note that generating a widget can add additional latency to the response, so it is recommended that you do not enable the widget if you are not displaying it.

Assuming you have a Google Maps API key with both APIs enabled, the following code shows one way to render the widget.

In [ ]:
from IPython.display import HTML

# Load or set your Maps API key here.
MAPS_API_KEY = userdata.get("MAPS_API_KEY")

# This is the same request as above, except `enable_widget` is set.
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Do any cafes around here do a good flat white? I will walk up to 20 minutes away",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_maps=types.GoogleMaps(enable_widget=True))],
        tool_config=types.ToolConfig(
            retrieval_config=types.RetrievalConfig(
                lat_lng=types.LatLng(latitude=40.7680797, longitude=-73.9818957)
            )
        ),
    ),
)

widget_token = response.candidates[0].grounding_metadata.google_maps_widget_context_token

display(Markdown(f"### Response\n {response.text}"))
display(Markdown(generate_sources(response)))
display(HTML(f"""
<!DOCTYPE html>
<html>
  <body>
    <div style="max-width: 500px; margin: 0 auto">
      <script src="https://maps.googleapis.com/maps/api/js?key={MAPS_API_KEY}&loading=async&v=alpha&libraries=places" async></script>
      <gmp-place-contextual context-token="{widget_token}"></gmp-place-contextual>
    </div>
  </body>
</html>
"""))

### Response
 There are several highly-rated cafes within a 20-minute walk that serve coffee.

If you're looking for a café open right now, **Heaven on 7th Marketplace** is open 24 hours, has a 4.8-star rating, and is approximately a 2.7-minute walk (576 meters) away. They serve coffee and smoothies along with sandwiches and bagels.

For a café that explicitly mentions flat whites and has a high rating, **Tiny Dancer Coffee** is an excellent option, rated 4.8 stars. They serve espressos and flat whites, as well as oat and matcha latte options. It's about a 6.8-minute walk (1.3 kilometers) away and opens at 7:00 AM local time.

Other well-rated cafes that open soon and are within a short walk include:

*   **Cafe aroma**, with a 4.7-star rating, opens at 6:30 AM and is a 1.6-minute walk (279 meters) away. They offer hot drinks along with bagels, sandwiches, and pastries.
*   **Down Under Coffee**, rated 4.8 stars, opens at 7:30 AM and is a 1.9-minute walk (321 meters) away.
*   **Masseria Caffè**, a 4.6-star rated café, opens at 7:00 AM and is a 2.3-minute walk (472 meters) away. They offer a variety of caffeinated beverages and pastries.
*   **Weill Café**, boasting a 4.9-star rating, opens at 8:00 AM and is a very short 1.7-minute walk (425 meters) away.

### Sources from Google Maps
- [White Noise Coffee - Coffee Shop & Roastery](https://maps.google.com/?cid=9563404650783060353)
- [Tiny Dancer Coffee](https://maps.google.com/?cid=14421445427760414557)
- [Weill Café](https://maps.google.com/?cid=16521712104323291061)
- [maman](https://maps.google.com/?cid=14208928559726348633)
- [Bibble & Sip](https://maps.google.com/?cid=5234372605966457616)

<IPython.core.display.HTML object>

Running and rendering the above code will require a Maps API key. Once you have it working, the widget will look like this.

![Rendered contextual Places widget](https://storage.googleapis.com/generativeai-downloads/images/maps-widget.png)

<a name="yt_links"></a>

## Grounding with YouTube links

You can directly include a public YouTube URL in your prompt. The Gemini models will then process the video content to perform tasks like summarization and answering questions about the content.

This capability leverages Gemini's multimodal understanding, allowing it to analyze and interpret video data alongside any text prompts provided.

You do need to explicitly declare the video URL you want the model to process as part of the contents of the request using a `FileData` part. Here a simple interaction where you ask the model to summarize a YouTube video:

In [ ]:
yt_link = "https://www.youtube.com/watch?v=XV1kOFo1C8M"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=types.Content(
        parts=[
            types.Part(text="Summarize this video."),
            types.Part(file_data=types.FileData(file_uri=yt_link)),
        ]
    ),
)

Markdown(response.text)

This video introduces "Gemma Chess," demonstrating how Google's large language model, Gemma, can enhance the game of chess by leveraging its linguistic abilities.

The speaker, Ju-yeong Ji from Google DeepMind, explains that Gemma isn't intended to replace powerful chess engines that excel at calculating moves. Instead, it aims to bring a "new dimension" to chess through understanding and creating text.

The video highlights three key applications:

1.  **Explainer:** Gemma can analyze chess games (e.g., Kasparov vs. Deep Blue) and explain the "most interesting" or strategically significant moves in plain language, detailing their impact, tactical considerations, and psychological aspects, making complex analyses more understandable.
2.  **Storytellers:** Gemma can generate narrative stories about chess games, transforming raw move data into engaging accounts that capture the tension, emotions, and key moments of a match, bringing the game to life beyond just the moves.
3.  **Supporting Chess Learning:** Gemma can act as a personalized chess tutor, explaining concepts like specific openings (e.g., Sicilian Defense) or tactics in an accessible way, even adapting to the user's language and skill level, effectively serving as an always-available, intelligent chess encyclopedia and coach.

By combining the computational strength of traditional chess AI with Gemma's advanced language capabilities, this approach offers a more intuitive and human-friendly way to learn, analyze, and engage with chess.

But you can also use the link as the source of truth for your request. In this example, you will first ask how Gemma models can help on chess games:

In [ ]:
yt_link = "https://www.youtube.com/watch?v=XV1kOFo1C8M"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=types.Content(
        parts=[
            types.Part(
                text="In 2 paragraph, how Gemma models can help on chess games?"
            ),
            types.Part(file_data=types.FileData(file_uri=yt_link)),
        ]
    ),
)

Markdown(response.text)

Gemma models, as large language models (LLMs), can significantly enhance the chess experience by bridging the gap between raw computational power and human understanding. Unlike traditional chess engines that excel at brute-force calculation and generating optimal moves (often in cryptic notation or complex numerical evaluations), Gemma's strength lies in processing and generating human-like text. This allows it to translate intricate chess engine outputs into intuitive, prose-based explanations, elucidating the strategic and tactical rationale behind moves, clarifying complex game concepts like openings and endgames, and providing accessible insights for players of all skill levels, significantly enhancing understanding beyond mere data.

Furthermore, Gemma can serve as an invaluable tool for personalized chess learning and engagement. It can act as a dynamic, interactive coach, offering tailored explanations of specific positions, identifying weaknesses in a player's understanding, or even detailing the historical and psychological context of famous matches. By summarizing complex game analyses, highlighting pivotal moments, and even crafting narrative descriptions of entire games, Gemma can make chess more approachable, immersive, and educational, transforming how players learn, analyze, and appreciate the strategic depth of the game.

Now your answer is more insightful for the topic you want, using the knowledge shared on the video and not necessarily available on the model knowledge.

<a name="url_context"></a>

## Grounding information using URL context

The URL Context tool empowers Gemini models to directly access and process content from specific web page URLs you provide within your API requests. This is incredibly interesting because it allows your applications to dynamically interact with live web information without needing you to manually pre-process and feed that content to the model.

URL Context is effective because it allows the models to base its responses and analysis directly on the content of the designated web pages. Instead of relying solely on its general training data or broad web searches (which are also valuable grounding tools), URL Context anchors the model's understanding to the specific information present at those URLs.

### Process website URLs

If you want Gemini to specifically ground its answers thanks to the content of a specific website, just add the urls in your prompt and enable the tool by adding it to your config:
```
config = {
  "tools": [
    {
      "url_context": {}
    }
  ],
}
```

You can add up to 20 links in your prompt.

In [ ]:
prompt = """
  Based on https://ai.google.dev/gemini-api/docs/models, what are the key
  differences between Gemini 1.5, Gemini 2.0 and Gemini 2.5 models?
  Create a markdown table comparing the differences.
"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text))

The provided document details various Gemini model variants, including Gemini 1.5, Gemini 2.0, and Gemini 2.5, each with different "Flash," "Pro," and "Lite" versions optimized for specific use cases.

Here's a comparison of the key differences:

| Feature           | Gemini 1.5 Pro                                  | Gemini 1.5 Flash                                  | Gemini 2.0 Flash                                     | Gemini 2.5 Pro                                                                  | Gemini 2.5 Flash                                                            | Gemini 2.5 Flash-Lite                                                     |
| :---------------- | :---------------------------------------------- | :------------------------------------------------ | :--------------------------------------------------- | :------------------------------------------------------------------------------ | :-------------------------------------------------------------------------- | :------------------------------------------------------------------------ |
| **Description**   | Mid-size multimodal model, optimized for reasoning tasks, can process large amounts of data. | Fast and versatile multimodal model for diverse tasks. | Next-gen features, improved capabilities, superior speed, and native tool use. | Most powerful thinking model, maximum accuracy, state-of-the-art performance. | Best model in terms of price-performance, well-rounded capabilities. | Optimized for cost-efficiency and high throughput. |
| **Input(s)**      | Audio, images, video, text.                 | Audio, images, video, text.                   | Audio, images, video, text.                      | Audio, images, video, text, and PDF.                                        | Audio, images, video, and text.                                       | Text, image, video, audio.                                            |
| **Output(s)**     | Text.                                       | Text.                                         | Text.                                            | Text.                                                                       | Text.                                                                   | Text.                                                                 |
| **Input Token Limit** | 2,097,152.                                  | 1,048,576.                                    | 1,048,576.                                       | 1,048,576.                                                                  | 1,048,576.                                                              | 1,048,576.                                                            |
| **Output Token Limit** | 8,192.                                      | 8,192.                                        | 8,192.                                           | 65,536.                                                                     | 65,536.                                                                 | 65,536.                                                               |
| **Key Use Cases** | Complex reasoning tasks.                    | Scaling across diverse tasks.                 | Next generation features, speed, realtime streaming. | Complex coding, reasoning, multimodal understanding, analyzing large data.  | Low latency, high volume tasks that require thinking.                   | Real time, low latency use cases.                                     |
| **Thinking**      | Not explicitly mentioned as a core capability, but optimized for reasoning tasks. | Not explicitly mentioned.                     | Experimental.                                    | Supported (default on).                                                     | Supported (default on, can configure thinking budget).                  | Supported.                                                            |
| **Live API**      | Not supported.                              | Not supported.                                | Supported.                                       | Not supported.                                                              | Not explicitly mentioned for the base Flash model, but Live variants exist. | Not supported.                                                        |
| **Knowledge Cutoff** | September 2024.                             | September 2024.                               | August 2024.                                     | January 2025.                                                               | January 2025.                                                           | January 2025.                                                         |
| **Deprecation** | September 2025.                             | September 2025.                               | Not deprecated.                                  | Not deprecated.                                                             | Not deprecated.                                                         | Not deprecated.                                                       |


You can see the status of the retrival using `url_context_metadata`:

In [ ]:
# get URLs retrieved for context
print(response.candidates[0].url_context_metadata)

url_metadata=[UrlMetadata(
  retrieved_url='https://ai.google.dev/gemini-api/docs/models',
  url_retrieval_status=<UrlRetrievalStatus.URL_RETRIEVAL_STATUS_SUCCESS: 'URL_RETRIEVAL_STATUS_SUCCESS'>
)]


### Add PDFs by URL

Gemini can also process PDFs from an URL. Here's an example:

In [7]:
# =========================
# ONE CELL: MULTI PDF + OCR + RETRY + TRIM + SAVE PDFNAME.md
# + BATCH MODE + RESUME USING done.log
# =========================

!apt-get -qq update
!apt-get -qq install -y tesseract-ocr poppler-utils
!pip -q install pypdf pytesseract pdf2image pillow

import os, re, time, random
from google.colab import files
from pypdf import PdfReader
from pdf2image import convert_from_path
import pytesseract
from IPython.display import Markdown, display


# =========================
# 0) SETTINGS (CHANGE THESE)
# =========================
BATCH_SIZE = 30          # process only 30 PDFs per run (safe)
COOLDOWN_SECONDS = 2     # delay between model calls
MAX_PROMPT_CHARS = 35000 # prevents token overload
OCR_MAX_PAGES = 3        # OCR only first 3 pages (fast)


# =========================
# 1) UPLOAD MULTIPLE PDFs
# =========================
uploaded = files.upload()   # select many PDFs


# =========================
# 2) SAFE FILE NAME
# =========================
def safe_name(name):
    name = os.path.splitext(name)[0]
    name = re.sub(r"[^a-zA-Z0-9._-]+", "_", name)
    return name[:90].strip("_")


# =========================
# 3) OUTPUT ROOT + LOG
# =========================
OUTPUT_ROOT = "outputs"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

DONE_LOG = os.path.join(OUTPUT_ROOT, "done.log")

done_set = set()
if os.path.exists(DONE_LOG):
    with open(DONE_LOG, "r", encoding="utf-8") as f:
        done_set = set(line.strip() for line in f if line.strip())


# =========================
# 4) EXTRACT TEXT (NORMAL PDF)
# =========================
def extract_text_pypdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = []
    for page in reader.pages:
        t = page.extract_text()
        if t:
            text.append(t)
    return "\n".join(text).strip()


# =========================
# 5) OCR (SCANNED PDF) - LIMITED PAGES (FAST)
# =========================
def extract_text_ocr_limited(pdf_path, dpi=200, max_pages=OCR_MAX_PAGES):
    images = convert_from_path(pdf_path, dpi=dpi, first_page=1, last_page=max_pages)
    out = []
    for i, img in enumerate(images, start=1):
        page_text = pytesseract.image_to_string(img)
        if page_text.strip():
            out.append(f"\n\n--- OCR PAGE {i} ---\n{page_text}")
    return "\n".join(out).strip()


# =========================
# 6) SMART EXTRACTOR (AUTO)
# =========================
def extract_pdf_text_smart(pdf_path):
    text = extract_text_pypdf(pdf_path)

    # If text exists -> no OCR
    if len(text) >= 800:
        print(f"[INFO] {pdf_path}: Text-based PDF → normal extraction used.")
        return text

    # Otherwise -> OCR limited pages
    print(f"[INFO] {pdf_path}: Low text → OCR first {OCR_MAX_PAGES} pages...")
    return extract_text_ocr_limited(pdf_path)


# =========================
# 7) TRIM TEXT (TOKEN SAFETY)
# =========================
def trim_pdf_text(pdf_text, max_chars=MAX_PROMPT_CHARS):
    pdf_text = pdf_text.strip()

    if len(pdf_text) <= max_chars:
        return pdf_text

    head = pdf_text[:14000]
    mid_start = len(pdf_text) // 2 - 4000
    mid_end = len(pdf_text) // 2 + 4000
    mid = pdf_text[mid_start:mid_end]
    tail = pdf_text[-14000:]

    return (
        "----- PDF TEXT TRIMMED FOR TOKEN SAFETY -----\n\n"
        "[START OF PDF]\n" + head +
        "\n\n[MIDDLE OF PDF]\n" + mid +
        "\n\n[END OF PDF]\n" + tail +
        "\n\n----- END TRIM -----"
    )


# =========================
# 8) MASTER PROMPT (PASTE YOUR FULL PROMPT HERE)
# =========================
MASTER_PROMPT = r"""
# MASTER SYSTEM PROMPT
(Job Notification Intelligence Engine – Decision-Grade, User-First Ordered Version)

---

## SYSTEM / INSTRUCTION PROMPT

You are a **Job Notification Intelligence Engine**.

Your task is to analyze the given job notification PDF text and extract **structured, decision-grade, candidate-segmented information** that helps eligible candidates clearly understand:

* Whether they can apply
* How to apply
* Deadlines
* Category-wise seat availability
* Disqualifiers and deal-breakers
* Whether the job is worth applying for

Accuracy is mandatory. Follow all rules strictly.

---

## INPUT

You will be given **raw text extracted from a job notification PDF** (may include OCR noise, formatting loss, broken tables, or repeated lines).

---

## CORE PRINCIPLES (NON-NEGOTIABLE)

1. Do NOT assume any personal profile of the user.
2. ONLY create candidate profiles that are explicitly eligible according to the PDF.
3. Do NOT invent qualifications, posts, categories, reservations, salary, experience, dates, fees, or locations.
4. Do NOT include ineligible or hypothetical profiles.
5. If information is missing, explicitly state:
   **"Not specified in notification"**
6. If category-wise vacancy is missing, explicitly state:
   **"Category-wise breakup not specified in the notification"**
7. Never summarize generically — always structure.
8. Never infer silently — flag uncertainty.
9. Detect and explicitly flag silent eliminators (bond, credit score, locality, registration requirements, etc.).
10. If a section cannot be supported by PDF text, output:
    **"Insufficient information in notification"**
11. If the notification contains multiple posts, treat each post independently.
    Never merge eligibility, dates, fees, selection process, or vacancies across posts.
12. Before listing any candidate profile as eligible, validate ALL hard gates explicitly mentioned in the PDF, including:

* Age limits and cutoff date
* Educational qualification rules
* Experience requirements (type, duration, sector)
* Nationality requirements
* Domicile/locality rules
* Category certificate validity rules
* Registration requirements (NAPS/NATS/Employment exchange/etc.)
  If any hard gate is unclear due to OCR noise or missing text, do NOT list the profile as eligible.

13. If tables exist, treat tables as the primary source of truth.

* Parse tables explicitly
* Preserve numeric accuracy exactly
* Never flatten tables into prose
* If paragraph text conflicts with a table, the table overrides.

14. Ambiguity firewall:
    If eligibility is ambiguous due to OCR noise or unclear phrasing, mark confidence as LOW and do NOT include that candidate profile unless eligibility is explicitly confirmed.
---

15. JSON Eligibility Objects (Mandatory)

At the end, output a strict JSON array named `json_eligibility_objects`.

Rules:
- One object per post.
- `post_name` must exactly match the post name in the notification.
- `eligibility_ids` must contain ALL keys below.
- No nulls allowed.
- Use `"not_specified"` where data is missing.
- Use arrays for multi-values.
- Use boolean true/false only for `domicile_required` and `experience_required`.
- Preserve exact eligibility meaning from the PDF.

Required keys inside `eligibility_ids`:

- min_age
- max_age
- gender_allowed
- nationality_required
- domicile_required
- domicile_state
- domicile_district
- domicile_block_or_area
- education_levels
- education_specializations
- experience_required
- experience_required_for
- minimum_experience_years
- experience_domain

Output format example:

```json
[
  {
    "post_name": "Post Name",
    "eligibility_ids": {
      "min_age": 0,
      "max_age": 0,
      "gender_allowed": ["all"],
      "nationality_required": "not_specified",
      "domicile_required": false,
      "domicile_state": "not_specified",
      "domicile_district": "not_specified",
      "domicile_block_or_area": "not_specified",
      "education_levels": ["not_specified"],
      "education_specializations": ["not_specified"],
      "experience_required": false,
      "experience_required_for": ["not_specified"],
      "minimum_experience_years": 0,
      "experience_domain": "not_specified"
    }
  }
]


---

## CONFIDENCE & TRACEABILITY RULES

For each major section:

* Include a **Confidence Level**:

  * High → explicitly stated in the PDF
  * Medium → clearly derived from explicit text
  * Low → implied, unclear, or OCR-corrupted (must be flagged)

* Include a **Source Anchor**:

  * Quote or clause from PDF text that supports the extracted data
  * If table-based, cite the table row/heading content as the anchor

---

## NORMALIZATION RULES (FOR API / SAAS BACKENDS)

When extractable:

* Dates normalized to: `YYYY-MM-DD`
* Currency preserved exactly as written, and also converted into numeric INR when possible
* Age structured as:

  * `min_age`, `max_age`, `age_cutoff_date`
* Vacancy counts preserved exactly (no rounding, no assumptions)

If any normalized value cannot be reliably extracted, output:
**"Not specified in notification"**

---

# REQUIRED OUTPUT STRUCTURE

(Use Markdown. Do NOT add explanations outside this structure.)

---

## 1) Job Overview (Fast Decision Snapshot)

* Organization Name:
* Job Title(s):
* Total Vacancies:
* Job Type: (Permanent / Contract / Temporary)
* Posting Location(s):

Confidence Level:
Source:

---

## 2) Important Dates + Application Status (Critical)

* Application Start:
* Application End:
* Exam:
* Interview:

Application Status: Open / Closing Soon / Closed / Cannot be determined
Confidence Level:
Source:

---

## 3) Eligible Candidate Profiles (Segmented)

Only include profiles that are truly eligible as per the PDF.

---

### Candidate Type: `<Post Name + Minimum Qualification Group>`

#### Category-wise Seats

* SC:
* ST:
* OBC:
* EWS:
* UR:
* PwBD (if applicable):

(If not specified → state clearly)

Confidence Level:
Source:

---

#### Eligibility Criteria (Hard Gates)

* Education:
* Experience:
* Age Limit:
* Age Relaxation:
* Cutoff Date for Age/Eligibility:

Confidence Level:
Source:

---

#### Salary / Pay Scale

Confidence Level:
Source:

---

#### Selection Process

Confidence Level:
Source:

---

#### How to Apply

* Mode of Application:
* Official Website / Address:
* Documents Required:
* Application Fee:

Confidence Level:
Source:

---

#### Automatic Disqualification Conditions (Hard Eliminators)

Explicitly list conditions causing direct rejection:

* Credit score requirement
* Experience type restrictions
* Medical standards
* Bond/service agreement
* Locality/residence requirements
* Category certificate rules
* Registration requirements
* Multiple applications allowed? (Yes / No / Not specified)
* Multiple posts allowed? (Yes / No / Not specified)

Confidence Level:
Source:

---

#### Post-Selection Obligations

* Probation period:
* Bond amount & duration:
* Service conditions:
* Penalty for early exit:

(If not specified → mark clearly)

Confidence Level:
Source:

---

#### Real Salary Insight

* Approx in-hand per month:
* Major deductions:
* Benefits included:
* Benefits NOT included:

(If not specified → mark clearly)

Confidence Level:
Source:

---

#### Category Advantage Insight

* Whether reservation meaningfully improves chances
* Whether UR candidates are disadvantaged
* Seat-to-competition signal (only if inferable from explicit vacancy distribution or eligibility narrowing)

(If not inferable → state clearly)

---

---

## 4) Who Should Apply (Final Decision Guidance)

* Apply if:
* Do NOT apply if:

Confidence Level:
Source:

---

## 5) Risk Flags (Deal-breaker Warnings)

Examples:

* Low initial pay
* Long bond
* Contract job with no renewal
* High document rejection risk
* Location restriction

If none, output exactly:

"No explicit risk flags identified in notification."

Confidence Level:
Source:

---

## 6) Important Links

* Official Notification:
* Apply Online:
* Helpdesk / Contact:
* Admit Card (if mentioned):

Confidence Level:
Source:

---

## 7) Common Mistakes That Lead to Rejection

List only practical, realistic errors, such as:

* Incorrect photo/signature format
* Wrong category selection
* Incomplete certificates
* Experience miscalculation
* Multiple applications

Confidence Level:
Source:

---

## 8) Document Readiness Checklist

* Educational certificates ✔ / ❌
* Category certificate ✔ / ❌
* Experience proof ✔ / ❌
* Photo & Signature ✔ / ❌
* ID proof ✔ / ❌
* Medical/Fitness certificate (if required) ✔ / ❌

---

## 9) Missing or Unclear Information

Explicitly list any important data not clearly stated:

* Salary breakup
* Syllabus
* Transfer policy
* Promotion rules
* Posting policy

---

## 10) Ineligible Profiles (Explicitly Excluded by PDF)

List profiles that clearly cannot apply, with reason.

Confidence Level:
Source:

---

## 11) Secondary Analysis (Optional but Useful)

### Career Impact Analysis

* Nature of job: Entry-level / Growth / Specialist / Lateral
* Promotion scope:
* Transferability:
* Long-term career value:

Confidence Level:
Source:

---

### Competition Level

Estimate based ONLY on factors supported by the notification:

* Vacancy count
* Eligibility width
* National vs local scope
* Exam stages and difficulty indicators (if described)

Classification: Low / Medium / High
Confidence Level:
Source:

---

### Application Effort Score

* Form complexity: Low / Medium / High
* Number of documents required:
* Exam involved: Yes / No
* Interview involved: Yes / No
* Overall effort: Low / Medium / High

Confidence Level:
Source:

---

## 12) Extraction Quality Report (Mandatory for Reliability)

* OCR quality: Good / Medium / Poor
* Table extraction reliability: High / Medium / Low
* Missing critical fields: list
* Risk of wrong eligibility extraction: Low / Medium / High

---

## OUTPUT FORMAT RULES

* Use Markdown
* Use headings & bullet points
* No emojis
* No assumptions
* No filler text
* Suitable for both humans and machines

---

## OPTIONAL (SYSTEM / API MODE)

Also generate a strict JSON output following the same structure:

* snake_case keys
* no nulls (use "not_specified")
* preserve candidate segmentation exactly
* preserve all vacancy numbers exactly

---

## INPUT PLACEHOLDER

<<PASTE PDF TEXT HERE>>
"""



# =========================
# 9) MODEL CALL WITH RETRY (503 SAFE)
# =========================
config = {
    "tools": [{"url_context": {}}],
}

def generate_with_retry(final_prompt, max_retries=7):
    for attempt in range(1, max_retries + 1):
        try:
            return client.models.generate_content(
                contents=[final_prompt],
                model=MODEL_ID,
                config=config
            )
        except Exception as e:
            msg = str(e)

            if "503" in msg or "overloaded" in msg or "UNAVAILABLE" in msg:
                wait = (2 ** attempt) + random.uniform(0, 2)
                print(f"[WARN] Model overloaded (attempt {attempt}/{max_retries}). Waiting {wait:.1f}s...")
                time.sleep(wait)
                continue

            raise e

    return None


# =========================
# 10) PROCESS IN BATCH + RESUME
# =========================
all_pdfs = [f for f in uploaded.keys() if f.lower().endswith(".pdf")]

pending = []
for pdf in all_pdfs:
    md_filename = safe_name(pdf) + ".md"
    md_path = os.path.join(OUTPUT_ROOT, md_filename)

    # Skip if already processed
    if pdf in done_set or os.path.exists(md_path):
        continue

    pending.append(pdf)

print("\n" + "=" * 90)
print(f"TOTAL UPLOADED PDFs   : {len(all_pdfs)}")
print(f"ALREADY PROCESSED     : {len(all_pdfs) - len(pending)}")
print(f"PENDING THIS SESSION  : {len(pending)}")
print(f"PROCESSING THIS BATCH : {min(BATCH_SIZE, len(pending))}")
print("=" * 90)


processed_now = 0

for filename in pending[:BATCH_SIZE]:
    print("\n" + "=" * 90)
    print("PROCESSING:", filename)
    print("=" * 90)

    try:
        pdf_text = extract_pdf_text_smart(filename)

        if not pdf_text.strip():
            print("[ERROR] No text extracted even after OCR.")
            continue

        pdf_text = trim_pdf_text(pdf_text)

        final_prompt = MASTER_PROMPT.replace("<<PASTE PDF TEXT HERE>>", pdf_text)

        response = generate_with_retry(final_prompt)

        if response is None:
            print("[SKIPPED] Model overloaded too long. Skipping this PDF.")
            continue

        result_text = response.text.strip()

        md_filename = safe_name(filename) + ".md"
        md_path = os.path.join(OUTPUT_ROOT, md_filename)

        with open(md_path, "w", encoding="utf-8") as f:
            f.write(result_text)

        # Mark as done in log
        with open(DONE_LOG, "a", encoding="utf-8") as f:
            f.write(filename + "\n")

        print(f"[SAVED] {md_path}")
        display(Markdown(result_text.replace("$", "\\$")))

        processed_now += 1
        time.sleep(COOLDOWN_SECONDS)

    except Exception as e:
        print("[ERROR] Failed for:", filename)
        print("Reason:", str(e))
        continue


print("\n" + "=" * 90)
print(f"[DONE] Processed this batch: {processed_now}")
print("All results are saved inside:", OUTPUT_ROOT)
print("Next run will automatically resume using done.log")
print("=" * 90)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Saving MAHADISCOM_MAHADISCOM___Additional_Execut_Official_Notification_PDF_2026-02-04_2026-02-04T07-47-32.pdf to MAHADISCOM_MAHADISCOM___Additional_Execut_Official_Notification_PDF_2026-02-04_2026-02-04T07-47-32.pdf

TOTAL UPLOADED PDFs   : 1
ALREADY PROCESSED     : 0
PENDING THIS SESSION  : 1
PROCESSING THIS BATCH : 1

PROCESSING: MAHADISCOM_MAHADISCOM___Additional_Execut_Official_Notification_PDF_2026-02-04_2026-02-04T07-47-32.pdf
[INFO] MAHADISCOM_MAHADISCOM___Additional_Execut_Official_Notification_PDF_2026-02-04_2026-02-04T07-47-32.pdf: Text-based PDF → normal extraction used.
[SAVED] outputs/MAHADISCOM_MAHADISCOM___Additional_Execut_Official_Notification_PDF_2026-02-04_2026-02-04T.md


## 1) Job Overview (Fast Decision Snapshot)

* **Organization Name:** Maharashtra State Electricity Distribution Co. Ltd (MSEDCL / Mahadiscom)
* **Job Title(s):** 
    1. Additional Executive Engineer (Distribution)
    2. Additional Executive Engineer (Civil)
    3. Deputy Executive Engineer (Distribution)
    4. Deputy Executive Engineer (Civil)
* **Total Vacancies:** 180 (94 + 5 + 69 + 12)
* **Job Type:** Permanent (subject to 3-year service bond)
* **Posting Location(s):** Maharashtra State (specific locations determined after selection)

**Confidence Level:** High
**Source:** Page 1-2, Section 1 (Number of Vacancies) and Page 22, clause xxiii.

---

## 2) Important Dates + Application Status (Critical)

* **Notification Date:** 2025-06-27
* **Application Start:** Not specified in notification (Snippet covers registration procedure but not dates)
* **Application End:** Not specified in notification
* **Exam:** Not specified in notification
* **Interview:** Not specified in notification

**Application Status:** Cannot be determined (Specific dates missing from provided text)
**Confidence Level:** Low (for specific timeline) / High (for cutoff date)
**Source:** Advertisement No. 02/2025 Date. Cutoff date for age and experience is explicitly 2025-06-27.

---

## 3) Eligible Candidate Profiles (Segmented)

### Candidate Type 1: Additional Executive Engineer (Dist.)
#### Category-wise Seats
* **SC:** 9
* **ST:** 6
* **VJ(A):** 4
* **NT(B):** 2
* **NT(C):** 4
* **NT(D):** 1
* **SBC:** 2
* **OBC:** 13
* **SEBC:** 6
* **OPEN:** 47 (Includes EWS clubbed posts)
* **PwBD:** 4 (Horizontal)
* **Total:** 94

**Confidence Level:** High
**Source:** Page 1, Table (i)

#### Eligibility Criteria (Hard Gates)
* **Education:** Bachelor’s Degree in Electrical Engineering/Technology.
* **Experience:** 
    * 07 years post-qualification experience in Power Sector.
    * Must include 5 years in Power Distribution at the level of Asst. Engineer or above.
    * Must include 1 year as Dy. Executive Engineer (Dist.) OR 2 years as Dy. Executive Engineer (Dist.) within the 7 total years.
* **Age Limit:** 40 Years.
* **Age Relaxation:** 5 years for BC/Orphan/Sports; Up to 45 for PwBD; Up to 57 for departmental candidates.
* **Cutoff Date for Age/Eligibility:** 2025-06-27

**Confidence Level:** High
**Source:** Page 2-4, Section 2 & 3.

#### Salary / Pay Scale
* Rs. 81850 - 3250 - 98100 - 3455 - 184475 (Pay Group-I)

**Confidence Level:** High
**Source:** Page 4, Section 4.

---

### Candidate Type 2: Deputy Executive Engineer (Dist.)
#### Category-wise Seats
* **SC:** 10
* **ST:** 3
* **VJ(A):** 3
* **NT(B):** 1
* **NT(C):** 2
* **NT(D):** 1
* **SBC:** 2
* **OBC:** 15
* **SEBC:** 5
* **OPEN:** 27
* **PwBD:** 3 (Horizontal)
* **Total:** 69

**Confidence Level:** High
**Source:** Page 2, Table (iii)

#### Eligibility Criteria (Hard Gates)
* **Education:** Bachelor’s Degree in Electrical Engineering/Technology.
* **Experience:** 3 years post-qualification experience in Power Distribution as Junior Engineer (Dist.) or above, including 1 year as Asst. Engineer (Dist.).
* **Age Limit:** 35 Years.
* **Cutoff Date:** 2025-06-27

**Confidence Level:** High
**Source:** Page 2-4, Section 2 & 3.

#### Salary / Pay Scale
* Rs. 73580 - 2995 - 88555 - 3250 - 166555 (Pay Group-II)

**Confidence Level:** High
**Source:** Page 4, Section 4.

---

### Candidate Type 3: Additional Executive Engineer (Civil)
* **Total Vacancies:** 5 (SC:0, ST:0, VJ(A):0, NT(B):1, NT(C):0, NT(D):0, SBC:0, OBC:1, SEBC:1, OPEN:2)
* **Education:** Bachelor’s Degree in Civil Engineering/Technology.
* **Experience:** 7 years in Civil works related to Power Sector (including specific AE/DyEE level roles).
* **Age Limit:** 40 Years.
* **Pay Scale:** Rs. 81850 - 184475.

---

### Candidate Type 4: Deputy Executive Engineer (Civil)
* **Total Vacancies:** 12 (SC:2, ST:1, NT(B):1, OBC:3, SEBC:1, OPEN:4)
* **Education:** Bachelor’s Degree in Civil Engineering/Technology.
* **Experience:** 3 years in Civil works in Power Sector, including 1 year as Asst. Engineer (Civil).
* **Age Limit:** 35 Years.
* **Pay Scale:** Rs. 73580 - 166555.

---

#### Selection Process
* Online Test (English, except Marathi language test) and Personal Interview.

**Confidence Level:** High
**Source:** Page 21, Section vii & xxi.

#### How to Apply
* **Mode of Application:** Online Only.
* **Official Website:** www.mahadiscom.in
* **Documents Required:** Scanned Photo (20-50kb), Signature (Black ink, 10-20kb), Left Thumb Impression, Handwritten Declaration, Experience Certificates, Marathi Language Certificate.
* **Application Fee:** Not specified in provided notification snippet.

**Confidence Level:** Medium (Fees missing)
**Source:** Page 12-14, Section 10.

---

#### Automatic Disqualification Conditions (Hard Eliminators)
* **Locality/Residence:** Must be a Domicile of Maharashtra State.
* **Language:** Must be able to read, write, and speak Marathi fluently (Certificate required).
* **Small Family Rules:** Must comply with Maharashtra Civil Services (Declaration of Small Family) Rules, 2005.
* **Signature:** Signatures in CAPITAL LETTERS will be rejected.
* **Multiple Applications:** Only the latest valid application is accepted; fees for others forfeited.
* **Multiple Posts:** Allowed, but only one application per post is implied; multiple registrations for the *same* post result in only the latest being accepted.

**Confidence Level:** High
**Source:** Page 5 (Domicile), Page 21 (Marathi/Small Family), Page 12 (Multiple Applications).

---

#### Post-Selection Obligations
* **Probation period:** Not specified in notification.
* **Bond amount & duration:** 
    * Addl. EE: Rs. 1,00,000 (1 Lakh).
    * Dy. EE: Rs. 50,000.
    * Duration: Minimum 3 years service.
* **Penalty for early exit:** Payment of the respective bond amount.

**Confidence Level:** High
**Source:** Page 22, clause xxiv.

---

#### Real Salary Insight
* **Approx in-hand per month:** Not specified (Base pay is Rs. 81,850 and Rs. 73,580 respectively).
* **Benefits included:** DA, HRA, Medical Benefit, Leave Encashment, CPF, and Gratuity.

---

#### Category Advantage Insight
* **EWS Notice:** EWS candidates are explicitly warned that because the gross salary of these posts exceeds Rs. 8 Lakhs, EWS eligibility is unlikely. Consequently, EWS posts have been clubbed with the **OPEN** category.
* **Reservation:** Horizontal reservation (30% Women, 5% Sports) applies.

---

## 4) Who Should Apply (Final Decision Guidance)
* **Apply if:** You are a Civil/Electrical Engineer with at least 3-7 years of experience in the Power Sector, are a domicile of Maharashtra, and can prove Marathi language proficiency.
* **Do NOT apply if:** You do not have a Maharashtra Domicile, you cannot speak/write Marathi, or your experience is outside the Power Sector/Distribution.

---

## 5) Risk Flags (Deal-breaker Warnings)
* **Bond Requirement:** 3-year mandatory service or heavy penalty (up to 1 Lakh).
* **Marathi Language:** Strict requirement for certification or professor-signed attestation.
* **EWS Exclusion:** Candidates seeking EWS reservation will find no specific seats as they are merged with Open.

**Confidence Level:** High
**Source:** Page 5, 21, 22.

---

## 6) Important Links
* **Official Notification:** Available on www.mahadiscom.in
* **Apply Online:** Link within the "Careers" section of the official website.

---

## 7) Common Mistakes That Lead to Rejection
* Signature in Capital Letters.
* Failure to upload the specific handwritten declaration in the candidate's own writing.
* Not having a current-year Non-Creamy Layer (NCL) certificate (for VJ/NT/SBC/OBC/SEBC).
* Name mismatch between ID proof and application (critical for married female candidates).

---

## 8) Document Readiness Checklist
* Educational certificates (BE/B.Tech) ✔
* Category certificate (and NCL if applicable) ✔
* Experience proof (Detailed with roles/hierarchy) ✔
* Photo (4.5x3.5cm) & Signature (Black ink) ✔
* Marathi Language Certificate ✔
* Small Family Declaration ✔

---

## 9) Missing or Unclear Information
* **Application Fees:** The amounts are not mentioned in this snippet.
* **Syllabus:** Only mentioned as "Electrical Engineering" for departmental candidates; full syllabus not in text.
* **Tentative Exam Date:** Missing.

---

## 10) Ineligible Profiles (Explicitly Excluded)
* **Non-Maharashtra Domiciles:** Notification explicitly requires Maharashtra domicile.
* **Freshers:** All posts require a minimum of 3 years post-qualification experience.
* **Candidates without Marathi knowledge:** Proficiency is a mandatory selection criteria.

---

## 11) Secondary Analysis
* **Competition Level:** High (Specific experience requirements in the power sector limit the pool, but Mahadiscom is a highly sought-after employer).
* **Application Effort Score:** Medium (Requires detailed experience certificates and a specific handwritten declaration).

---

## 12) Extraction Quality Report
* **OCR quality:** Good.
* **Table extraction reliability:** High.
* **Missing critical fields:** Application Fees, Specific Application Open/Close dates.
* **Risk of wrong eligibility extraction:** Low.

---

## JSON Eligibility Objects

```json
[
  {
    "post_name": "Additional Executive Engineer (Dist.)",
    "eligibility_ids": {
      "min_age": 0,
      "max_age": 40,
      "gender_allowed": ["all"],
      "nationality_required": "Indian",
      "domicile_required": true,
      "domicile_state": "Maharashtra",
      "domicile_district": "not_specified",
      "domicile_block_or_area": "not_specified",
      "education_levels": ["Bachelor's Degree"],
      "education_specializations": ["Electrical Engineering", "Technology"],
      "experience_required": true,
      "experience_required_for": ["Power Sector", "Power Distribution"],
      "minimum_experience_years": 7,
      "experience_domain": "Power Distribution / Engineering"
    }
  },
  {
    "post_name": "Additional Executive Engineer (Civil)",
    "eligibility_ids": {
      "min_age": 0,
      "max_age": 40,
      "gender_allowed": ["all"],
      "nationality_required": "Indian",
      "domicile_required": true,
      "domicile_state": "Maharashtra",
      "domicile_district": "not_specified",
      "domicile_block_or_area": "not_specified",
      "education_levels": ["Bachelor's Degree"],
      "education_specializations": ["Civil Engineering", "Technology"],
      "experience_required": true,
      "experience_required_for": ["Civil works related to Power Sector"],
      "minimum_experience_years": 7,
      "experience_domain": "Civil Engineering"
    }
  },
  {
    "post_name": "Deputy Executive Engineer (Dist.)",
    "eligibility_ids": {
      "min_age": 0,
      "max_age": 35,
      "gender_allowed": ["all"],
      "nationality_required": "Indian",
      "domicile_required": true,
      "domicile_state": "Maharashtra",
      "domicile_district": "not_specified",
      "domicile_block_or_area": "not_specified",
      "education_levels": ["Bachelor's Degree"],
      "education_specializations": ["Electrical Engineering", "Technology"],
      "experience_required": true,
      "experience_required_for": ["Power Distribution"],
      "minimum_experience_years": 3,
      "experience_domain": "Power Distribution"
    }
  },
  {
    "post_name": "Deputy Executive Engineer (Civil)",
    "eligibility_ids": {
      "min_age": 0,
      "max_age": 35,
      "gender_allowed": ["all"],
      "nationality_required": "Indian",
      "domicile_required": true,
      "domicile_state": "Maharashtra",
      "domicile_district": "not_specified",
      "domicile_block_or_area": "not_specified",
      "education_levels": ["Bachelor's Degree"],
      "education_specializations": ["Civil Engineering", "Technology"],
      "experience_required": true,
      "experience_required_for": ["Civil works in Power Sector"],
      "minimum_experience_years": 3,
      "experience_domain": "Civil Engineering"
    }
  }
]
```


[DONE] Processed this batch: 1
All results are saved inside: outputs
Next run will automatically resume using done.log


In [ ]:
prompt = """
  Can you give me an overview of the content of this pdf?
  https://abc.xyz/assets/cc/27/3ada14014efbadd7a58472f1f3f4/2025q2-alphabet-earnings-release.pdf

"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text.replace("$", "\$")))



The PDF is Alphabet Inc.'s Second Quarter 2025 Earnings Release. It details the company's financial performance for the quarter ended June 30, 2025.

Key highlights include:
*   **Total Revenues:** Consolidated Alphabet revenues increased 14% year-over-year to \$96.4 billion.
*   **Google Services:** Revenues grew 12% to \$82.5 billion, driven by strong performance in Google Search & other, Google subscriptions, platforms, devices, and YouTube ads.
*   **Google Cloud:** Revenues increased 32% to \$13.6 billion, with growth in Google Cloud Platform (GCP) across core GCP products, AI Infrastructure, and Generative AI Solutions. Google Cloud's annual revenue run-rate is now over \$50 billion.
*   **Operating Income:** Total operating income rose 14%, and the operating margin was 32.4%.
*   **Net Income and EPS:** Net income increased 19%, and diluted EPS grew 22% to \$2.31.
*   **AI Impact:** CEO Sundar Pichai highlighted that AI is positively impacting every part of the business, driving strong momentum, with new features like AI Overviews and AI Mode performing well in Search.
*   **Capital Expenditures:** Alphabet plans to increase capital expenditures to approximately \$85 billion in 2025 due to strong demand for Cloud products and services.
*   **Issuance of Senior Unsecured Notes:** In May 2025, Alphabet issued \$12.5 billion in fixed-rate senior unsecured notes.

The document also provides detailed financial tables, including consolidated balance sheets, statements of income, and statements of cash flows, as well as segment results for Google Services, Google Cloud, and Other Bets. It also includes reconciliations of GAAP to non-GAAP financial measures.

### Add images by URL

Gemini can also process images from an URL. Here's an example:

In [ ]:
prompt = """
  Can you help me name of the numbered parts of that instrument, in French?
  https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Trombone.svg/960px-Trombone.svg.png

"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text))

I cannot directly interpret the numbered parts within the image you provided. However, I can give you the common names of trombone parts in French, which you can then match to the numbers on your image:

Here are some common parts of a trombone in French:
*   **Embouchure** (Mouthpiece)
*   **Pavillon** (Bell)
*   **Coulisse** (Slide)
*   **Coulisse d'accord** or **Pompe d'accord** (Tuning slide)
*   **Clé d'eau** or **Barillet** (Water key or spit valve)
*   **Entretoise** (Brace/Cross-stay, often used for various connecting rods)
*   **Manchon** (Ferrule/Sleeve, connecting parts)

Please match these names to the numbered parts in your image.

## Mix Search grounding and URL context

The different tools can also be use in conjunction by adding them both to the config. It's a good way to steer Gemini in the right direction and then let it do its magic using search grounding.

In [ ]:
prompt = """
  Can you give me an overview of the content of this pdf?
  https://abc.xyz/assets/cc/27/3ada14014efbadd7a58472f1f3f4/2025q2-alphabet-earnings-release.pdf
  Search on the web for the reaction of the main financial analysts, what's the trend?
"""

config = {
  "tools": [
      {"url_context": {}},
      {"google_search": {}}
  ],
}

response = client.models.generate_content(
  contents=[prompt],
  model=MODEL_ID,
  config=config
)

display(Markdown(response.text.replace('$','\$')))
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Alphabet Inc. announced strong financial results for the second quarter of 2025, ending June 30, 2025. Consolidated revenues increased by 14% year-over-year to \$96.4 billion, or 13% in constant currency, with double-digit growth seen across Google Search & other, YouTube ads, Google subscriptions, platforms, and devices, and Google Cloud.

Key financial highlights include:
*   **Total revenues** of \$96.428 billion, up from \$84.742 billion in Q2 2024.
*   **Net income** increased by 19% to \$28.196 billion.
*   **Diluted EPS** rose by 22% to \$2.31.
*   **Operating income** increased by 14% to \$31.271 billion, with an **operating margin** of 32.4%.
*   **Google Services revenues** increased by 12% to \$82.5 billion.
*   **Google Cloud revenues** significantly increased by 32% to \$13.6 billion, driven by growth in Google Cloud Platform (GCP), AI Infrastructure, and Generative AI Solutions. Its annual revenue run-rate now exceeds \$50 billion.
*   The company announced an increase in **capital expenditures** to approximately \$85 billion for 2025 due to strong demand for Cloud products and services.

Sundar Pichai, CEO of Alphabet, highlighted the company's "standout quarter" with robust growth, attributing success to leadership in AI and rapid shipping. He noted the positive impact of AI across the business, strong momentum in Search (including AI Overviews and AI Mode), and continued strong performance in YouTube and subscriptions.

**Financial Analyst Reactions and Trends:**

Financial analysts generally maintain a positive outlook on Alphabet, with a majority (43 out of 55) recommending "buy" or "strong buy" ratings. However, the average target price has seen a slight decline from approximately \$215 in March to \$202.05, indicating increased uncertainty. Despite this, the current consensus suggests a potential upside of 11% from recent trading levels as of mid-July 2025.

Prior to the earnings release, analysts expected a moderation in growth for Q2 2025, with projected revenue of \$93.8 billion (+10.7% YoY) and net income of \$26.5 billion (+12.2% YoY). Alphabet's actual Q2 2025 results surpassed these expectations, with EPS of \$2.31 beating the forecasted \$2.17 and revenue of \$96.43 billion exceeding projections.

Despite the positive earnings beat, Alphabet's stock experienced a modest increase of 0.57% in after-hours trading, and in some instances, a slight decline (around 1.5%) post-announcement. This dip was primarily attributed to the company's decision to raise its 2025 capital expenditures guidance by \$10 billion to \$85 billion, reflecting increased investments in AI and technology infrastructure, which raised some investor concerns about higher costs.

The key areas of focus for analysts include Alphabet's ability to expand its GenAI model's user base without impacting traditional Search revenue, and the continued growth of Google Cloud, which is seen as the company's largest growth opportunity. Analysts are closely monitoring AI developments, cloud growth, and regulatory challenges. The overall trend appears to be one of cautious optimism, with strong underlying business performance balanced by increased investment needs for future growth in AI and cloud services.

## Next steps

<a name="next_steps"></a>

* For more details about using Google Search grounding, check out the [Search Grounding cookbook](./Search_Grounding.ipynb).
* If you are looking for another scenarios using videos, take a look at the [Video understanding cookbook](./Video_understanding.ipynb).

Also check the other Gemini capabilities that you can find in the [Gemini quickstarts](https://github.com/google-gemini/cookbook/tree/main/quickstarts/).